<a href="https://colab.research.google.com/github/MevrouwHelderder/final_assignment/blob/main/Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is your final assignment. You'll get a lot of freedom in doing this assignment but that also means you have to make choices and explain the reasoning behind those choices in your report.

For this assignment you can use any dataset you can find from the [Our World in Data website](https://ourworldindata.org/).

Please formulate an answer to the following three questions in your report.

* **What is the biggest predictor of a large CO2 output per capita of a country?**
* **which countries are making the biggest strides in decreasing CO2 output?**
* **which non-fossil fuel energy technology will have the best price in the future?**




---


**1: Biggest predictor of CO2 output**

To determine this you may want to consider things like GDP per capita, diets, number of cars per capita, various energy source, mobility and other factors.

Your answer can also be a specific combination of certain factors.


---


**2: Biggest strides in decreasing CO2 output**

You'll need to find the relative CO2 output for each country to be able to calculate this. But countries can have growing and shrinking populations too, so it's probably a good idea to take this into account as well.


---


**3: Best future price for non-fossil fuel energy**

To be able to predict prices you'll probably need to use linear regression over the various non-fossil fuel options.


---


**Submitting your Assignment**

Once you're done with this module, you can go to the next item where you'll be able to submit your assignment.

Please submit both your written report and all notebooks you've created in creating the report. Make sure everything works before submitting.



# What contributes to CO2 output?


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext google.colab.data_table

In [25]:
path_base = "https://raw.githubusercontent.com/MevrouwHelderder/final_assignment/main/"

paths = {
    "co2_greenhouse" : path_base + "owid-co2-data.csv",
    "energy_total" : path_base + "owid-energy-data.csv",
    "net_zero_pledge" : path_base + "net-zero-targets.csv",
    "agri_land_use_total" : path_base + "total-agricultural-area-over-the-long-term.csv",
    "grazing" : path_base + "grazing-land-use-over-the-long-term.csv",
    "cropland" : path_base + "cropland-use-over-the-long-term.csv"
}

dataframes = {}

for key, value in paths.items():
  dataframes[key] = pd.read_csv(value)

energy_total_df = dataframes["energy_total"]
co2_greenhouse_df = dataframes["co2_greenhouse"]
net_zero_pledge_df = dataframes["net_zero_pledge"]
agri_land_use_total_df = dataframes["agri_land_use_total"]
grazing_df = dataframes["grazing"]
cropland_df = dataframes["cropland"]

# Thoughts: 
**For the biggest predictor of a large CO2 output per capita of a country, you can use the following datasets:**

* CO2 emissions by country: This dataset provides historical CO2 emissions data for different countries over time.
* Energy consumption by fuel type: This dataset shows the energy consumption of different countries categorized by fossil fuels (coal, oil, gas), renewables, and other sources.
* Population by country: This dataset provides population data for different countries over time.

By combining these datasets, you can calculate CO2 emissions per capita and analyze the relationship between various factors like energy consumption, population, and CO2 emissions.

**To identify the countries making the biggest strides in decreasing CO2 output, you can use the following datasets:**

* CO2 emissions by country: This dataset will be useful again to track changes in CO2 emissions over time.
* Annual change in CO2 emissions: This dataset provides the annual percentage change in CO2 emissions for different countries.
* Renewable energy production: This dataset includes information on the production of renewable energy, such as solar, wind, hydro, and geothermal.

By analyzing the annual change in CO2 emissions and comparing it with the growth in renewable energy production, you can identify countries that are successfully reducing their CO2 output.

**For determining the non-fossil fuel energy technology with the best price in the future, you can consider the following datasets:**

* Costs of different energy technologies: This dataset provides information on the costs of various energy technologies, including solar, wind, hydro, nuclear, and others. It can help you analyze the trends and projections for future energy costs.
* Renewable energy capacity and investment: This dataset shows the installed capacity and investment trends in renewable energy sources, which can provide insights into the growth potential and competitiveness of different technologies.

By examining the cost trends and comparing the capacity and investment data, you can assess which non-fossil fuel energy technology is expected to have the most favorable price in the future.


**Find what countries do:**

* did they make any pledges
* what kind of energy are they using now


* What contributes to CO2 output?

* What are the prices for the different types of non-fossil energy?

* What happens when a country outsources everything?

* How about CO2 vs the other GHG (greenhouse gasses)? Are there countries where CO2 falls but the other rise?

# 1: Biggest predictor of CO2 output

# 2: Biggest strides in decreasing CO2 output

First let's look at what countries produces, import, export and how that changed through the years.

* **co2**: annual total production-based co2 emission. measured in million tonnes. Measured in million tonnes.
* **co2_growth_abs**: annual growth of production-based co2 emission. Measured in million tonnes.
* **co2_growth_prct**: annual percentage growth of production-based co2 emission. measured in million tonnes. Measured in million tonnes.
* **co2_per_capita**, annual total production-based emissions of carbon dioxide (CO₂) per capita, measured in tonnes per person. 

* **trade_co2**: annual net carbon dioxide (CO₂) emissions embedded in trade. The net of import or export via traded goods. Positive = importer of CO₂ emissions; Negative =  exporter. Measured in million tonnes.
* **consumption_co2**: total co2 minus emissions embedded in exports, plus emissions embedded in imports. Consumption > production = importer of CO₂ emissions; Consumption < Production =  exporter. Measured in million tonnes.
* **consumption_co2**: total co2 minus emissions embedded in exports per capita, plus emissions embedded in imports. Consumption > production = importer of CO₂ emissions; Consumption < Production =  exporter. Measured in tonnes per person. 

Note for later: 
also interesting: 
* what do we see when we look at rich vs poor (columns regarding GDP)
* what do we see when we look at the CO2 per energy unit.




In [70]:
explore_co2 = co2_greenhouse_df.loc[:, ["country", "year", "iso_code", "population", "co2", "co2_growth_abs", "co2_growth_prct", "co2_per_capita", "trade_co2", "consumption_co2", "consumption_co2_per_capita" ]]
explore_co2 = explore_co2.set_index("year")
explore_co2.index = pd.to_datetime(explore_co2.index, format="%Y", errors="coerce")

# show only the true countries, not the combinations like continents
explore_co2 = explore_co2[~explore_co2["iso_code"].isnull()]


# 2020 is the latest year where the data on trade is available. Let's look at the top ten of the different measurements: 

In [71]:
# Total production, not corrected for trade :
top_co2_2020 = explore_co2.loc["2020-01-01 00:00:00",["country", "co2"]].sort_values(by="co2", ascending = False).head(10)
top_co2_2020

,country,co2
year,,
2020-01-01,China,10956.213
2020-01-01,United States,4715.691
2020-01-01,India,2445.012
2020-01-01,Russia,1624.221
2020-01-01,Japan,1042.224
2020-01-01,Iran,729.978
2020-01-01,Saudi Arabia,661.193
2020-01-01,Germany,639.381
2020-01-01,Indonesia,609.786


In [72]:
# Total production per capita, not corrected for trade :
top_co2_pc_2020 = explore_co2.loc["2020-01-01 00:00:00",["country", "co2_per_capita"]].sort_values(by="co2_per_capita", ascending = False).head(10)
top_co2_pc_2020

,country,co2_per_capita
year,,
2020-01-01,Qatar,33.640
2020-01-01,Bahrain,25.451
2020-01-01,Brunei,23.890
2020-01-01,Trinidad and Tobago,23.553
2020-01-01,Kuwait,22.883
2020-01-01,United Arab Emirates,21.436
2020-01-01,New Caledonia,18.807
2020-01-01,Saudi Arabia,18.368
2020-01-01,Oman,15.959


In [73]:
# Total co2 for own consumption: 
top_co2_consumer_2020 = explore_co2.loc["2020-01-01 00:00:00",["country", "consumption_co2"]].sort_values(by="consumption_co2", ascending = False).head(10)
top_co2_consumer_2020

,country,consumption_co2
year,,
2020-01-01,China,10033.401
2020-01-01,United States,5197.398
2020-01-01,India,2276.881
2020-01-01,Russia,1360.149
2020-01-01,Japan,1187.141
2020-01-01,Germany,769.492
2020-01-01,South Korea,659.497
2020-01-01,Iran,658.446
2020-01-01,Saudi Arabia,657.944


In [74]:
# Total co2 for own consumption per capita: 
top_co2_consumer_pc_2020 = explore_co2.loc["2020-01-01 00:00:00", ["country", "consumption_co2_per_capita"]].sort_values(by="consumption_co2_per_capita", ascending = False).head(10)
top_co2_consumer_pc_2020

,country,consumption_co2_per_capita
year,,
2020-01-01,Qatar,25.771
2020-01-01,Singapore,24.238
2020-01-01,Brunei,22.550
2020-01-01,Kuwait,22.232
2020-01-01,United Arab Emirates,19.848
2020-01-01,Saudi Arabia,18.278
2020-01-01,Trinidad and Tobago,17.253
2020-01-01,Malta,16.612
2020-01-01,United States,15.471


# Interesting!
It shows that countries that produce a lot do not nessecarily use a lot and vise versa.

next step: who changed the best and worse over the past x years (50?)

In [76]:
energy_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21890 entries, 0 to 21889
Columns: 129 entries, country to wind_share_energy
dtypes: float64(126), int64(1), object(2)
memory usage: 21.5+ MB


In [75]:
energy_total_df

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21885,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,NaN,NaN,NaN,NaN,21.693,...,0.136,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21886,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,NaN,NaN,NaN,NaN,25.910,...,0.218,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21887,Zimbabwe,2019,ZWE,15354606.0,NaN,NaN,NaN,NaN,NaN,24.748,...,0.364,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21888,Zimbabwe,2020,ZWE,15669663.0,NaN,NaN,NaN,NaN,NaN,22.336,...,0.395,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


In [ ]:
# Renewables:
renew_list = ["wind", "solar", "hydro", "biofuel" ]

# Fossil:
fossil_list = ["coal", "oil", "gas", "other_renewable", "renewable" ]

# low_carbon:
low_carbon_list = ["low_carbon", "nuclear", "other_renewable", "renewable"  ]

In [ ]:
# next: turn these lists into columnnames
# check what exactly is summed up in the combined columns

Renewable energy is energy derived from natural sources that are replenished at a higher rate than they are consumed. 

"country", "year", "iso_code", "population", "gdp", "electricity_demand", 


**electricity_demand**. Electricity demand, measured in terawatt-hours
**electricity_generation**. Electricity generation, measured in terawatt-hours
**energy_per_capita**. Primary energy consumption per capita, measured in kilowatt-hours
**energy_cons_change_pct**. Annual percentage change in primary energy consumption
**energy_cons_change_twh**. Annual change in primary energy consumption, measured in terawatt-hours

**fossil_electricity**. 	Electricity generation from fossil fuels, measured in terawatt-hours. (Sum of coal, oil and gas.)
**fossil_elec_per_capita**. 	Per capita electricity generation from fossil fuels, measured in kilowatt-hours. 
**fossil_fuel_consumption**. 	Fossil fuel consumption, measured in terawatt-hours. 
**fossil_energy_per_capita**. Per capita fossil fuel consumption, measured in kilowatt-hours. 
**fossil_cons_change_pct**. 	Annual percentage change in fossil fuel consumption
**fossil_cons_change_twh**. 	Annual change in fossil fuel consumption, measured in terawatt-hours

**biofuel_electricity**. 	Electricity generation from biofuels, measured in terawatt-hours
**biofuel_elec_per_capita**. 	Per capita electricity generation from biofuels, measured in kilowatt-hours
**biofuel_consumption**. 	Primary energy consumption from biofuels
**biofuel_cons_per_capita**. 	Per capita primary energy consumption from biofuels
**biofuel_cons_change_pct**. 	Annual percentage change in biofuel consumption
**biofuel_cons_change_twh**. 	Annual change in biofuel consumption

**coal_electricity**. 		Electricity generation from coal
**coal_elec_per_capita**. 	Per capita electricity generation from coal
**coal_consumption**. 		Primary energy consumption from coal
**coal_cons_per_capita**. 	Per capita primary energy consumption from coal
**coal_cons_change_twh**. 	Annual change in coal consumption
**coal_cons_change_pct**. 	Annual percentage change in coal consumption

**gas_electricity**. 		Electricity generation from gas
**gas_elec_per_capita**. 	Per capita electricity generation from gas
**gas_consumption**. 		Primary energy consumption from gas
**gas_energy_per_capita**. 	Per capita primary energy consumption from gas
**gas_cons_change_twh**. 	Annual change in gas consumption
**gas_cons_change_pct**.  	Annual percentage change in gas consumption

**hydro_electricity**. 		Electricity generation from hydropower
**hydro_elec_per_capita**.	Per capita electricity generation from hydropower
**hydro_consumption**.		Primary energy consumption from hydropower
**hydro_energy_per_capita**.	Per capita primary energy consumption from hydropower
**hydro_cons_change_pct**.	Annual percentage change in hydropower consumption 
**hydro_cons_change_twh**.	Annual change in hydropower consumption

**oil_electricity**.		Electricity generation from oil
**oil_elec_per_capita**.	Per capita electricity generation from oil
**oil_consumption**.		Primary energy consumption from oil
**oil_energy_per_capita**.	Per capita primary energy consumption from oil
**oil_cons_change_twh**.	Annual change in oil consumption
**oil_cons_change_pct**.	Annual percentage change in oil consumption

**low_carbon_electricity**.	Electricity generation from low-carbon sources, measured in terawatt-hours. (Sum of renewables and nuclear.)
**low_carbon_elec_per_capita**.Per capita electricity generation from low-carbon sources, measured in kilowatt-hours
**low_carbon_consumption**.	Primary energy consumption from low-carbon sources, measured in terawatt-hours
**low_carbon_energy_per_capita**.Per capita primary energy consumption from low-carbon sources
**low_carbon_cons_change_twh**.Annual change in low-carbon energy consumption, measured in terawatt-hours
**low_carbon_cons_change_pct**.Annual percentage change in low-carbon energy consumption

**nuclear_electricity**.	Electricity generation from nuclear power
**nuclear_elec_per_capita**.	Per capita electricity generation from nuclear power
**nuclear_consumption**.	Primary energy consumption from nuclear power 
**nuclear_energy_per_capita**.Per capita primary energy consumption from nuclear
**nuclear_cons_change_twh**.	Annual change in nuclear consumption
**nuclear_cons_change_pct**.	Annual percentage change in nuclear consumption

**other_renewable_electricity**.Electricity generation from other renewables including biofuels
**other_renewable_elec_per_capita**.Per capita electricity generation from other renewables including biofuels. 
**other_renewable_consumption**.Primary energy consumption from other renewables including biofuels
**other_renewable_energy_per_capita**.Per capita primary energy consumption from other renewables including biofuels
**other_renewable_cons_change_twh**.Annual change in other renewables consumption
**other_renewable_cons_change_pct**.Annual percentage change in other renewables consumption

**renewable_electricity**.	Electricity generation from renewables including biofuels
**renewable_elec_per_capita**.Per capita electricity generation from renewables including biofuels. 
**renewable_consumption**.	Primary energy consumption from renewables including biofuels
**renewable_energy_per_capita**.Per capita primary energy consumption from renewables including biofuels
**renewable_cons_change_twh**.Annual change in renewables consumption
**renewable_cons_change_pct**.Annual percentage change in renewables consumption

**solar_electricity**.		Electricity generation from solar 
**solar_elec_per_capita**.	Per capita electricity generation from solar
**solar_consumption**.		rimary energy consumption from solar
**solar_energy_per_capita**.	er capita primary energy consumption from solar
**solar_cons_change_twh**.	Annual change in  solar consumption
**solar_cons_change_pct**.	Annual percentage change in  solar consumption

**wind_electricity**.		Electricity generation from wind
**wind_elec_per_capita**.	Per capita electricity generation from wind
**wind_consumption**.		rimary energy consumption from wind
**wind_energy_per_capita**.	Per capita primary energy consumption from wind
**wind_cons_change_twh**.	Annual change in wind consumption
**wind_cons_change_pct**.	Annual percentage change in wind consumption

**carbon_intensity_elec**. 	Carbon intensity of electricity production, measured in grams of carbon dioxide emitted per kilowatt-hour

# What are the safest and cleanest?

https://ourworldindata.org/safest-sources-of-energy#the-safest-energy-sources-are-also-the-cleanest